<a href="https://colab.research.google.com/github/sdinesh01/world-freedoms-viz/blob/main/Political_freedoms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data source: https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2022/2022-02-22/freedom.csv

---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
!pip install -U kaleido

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79.9 MB 1.2 MB/s 


In [ ]:
import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
from plotly import *
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import kaleido

In [ ]:
import io
import PIL

## Import and preprocess data

In [ ]:
data1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA340/assignments/Assignment 4/freedom.csv')
data1

,country,year,CL,PR,Status,Region_Code,Region_Name,is_ldc
0,Afghanistan,1995,7,7,NF,142,Asia,1
1,Afghanistan,1996,7,7,NF,142,Asia,1
2,Afghanistan,1997,7,7,NF,142,Asia,1
3,Afghanistan,1998,7,7,NF,142,Asia,1
4,Afghanistan,1999,7,7,NF,142,Asia,1
...,...,...,...,...,...,...,...,...
4974,Zimbabwe,2016,5,5,PF,2,Africa,0
4975,Zimbabwe,2017,5,6,NF,2,Africa,0
4976,Zimbabwe,2018,5,5,PF,2,Africa,0
4977,Zimbabwe,2019,5,5,PF,2,Africa,0


In [ ]:
# df for start-end plot
# subset data for years 1995 and 2020
data_sub3 = data1.loc[(data1['year'] == 1995) | (data1['year'] == 2020)]

# include countries that have data for both years
country_counts = data_sub3['country'].value_counts()
country_list = country_counts[country_counts > 1].index.tolist()
zip_data_df = data_sub3[data_sub3['country'].isin(country_list)]

#rename Côte d'Ivoire
zip_data_df.loc[zip_data_df.country=='CÃƒÂ´te dÃ¢â‚¬â„¢Ivoire','country'] = "Côte d'Ivoire"

#sort Political Rights values by ascending
zip_data_df2 = zip_data_df.sort_values(by='PR', ascending=False)

#rename countries with condensed name 
zip_data_df2['country'] = zip_data_df2['country'].replace(['United Kingdom of Great Britain and Northern Ireland',"Lao People's Democratic Republic", "Democratic People's Republic of Korea",
                                                           "Iran (Islamic Republic of)", 'Republic of Korea','Venezuela (Bolivarian Republic of)', 'Bolivia (Plurinational State of)',
                                                           'Micronesia (Federated States of)', 'Democratic Republic of the Congo', 'United Republic of Tanzania', 'Saint Vincent and the Grenadines'],
                                                          ['U.K.','Laos','North Korea','Iran','South Korea','Venezuela','Bolivia','Micronesia','DRC','Tanzania', 'St. Vincent & Grenadines'])
# turn 'year' column into strings for discrete symbolization
zip_data_df2= zip_data_df2.astype({"year": str})

#replace values in 'Status' column with long value
status = {'NF':'Not Free','F':'Free','PF':'Partially Free'}
zip_data_df2['Status'].replace(status, inplace=True)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## With Plotly GO

In [ ]:
data0 = {"x": [], "y": [], "colors": [], "years": []}
data1 = {"x": [], "y": [], "colors": [], "years": []}
data2 = {"x": [], "y": [], "colors": [], "years": []}
data3 = {"x": [], "y": [], "colors": [], "years": []}
data4 = {"x": [], "y": [], "colors": [], "years": []}

for country in zip_data_df2[zip_data_df2.Region_Name == 'Asia']['country']:
    data0["x"].extend(
        [
            zip_data_df2.loc[(zip_data_df2.year == '1995') & (zip_data_df2.country == country)]["PR"].values[0],
            zip_data_df2.loc[(zip_data_df2.year == '2020') & (zip_data_df2.country == country)]["PR"].values[0],
            None,
        ]
    )
    data0["y"].extend([country, country, None]),
    data0["colors"].extend(["#00677f", "#ADEFD1", "black"]),
    data0["years"].extend(["1995", "2020", None])

for country in zip_data_df2[zip_data_df2.Region_Name == 'Africa']['country']:
    data1["x"].extend(
        [
            zip_data_df2.loc[(zip_data_df2.year == '1995') & (zip_data_df2.country == country)]["PR"].values[0],
            zip_data_df2.loc[(zip_data_df2.year == '2020') & (zip_data_df2.country == country)]["PR"].values[0],
            None,
        ]
    )
    data1["y"].extend([country, country, None]),
    data1["colors"].extend(["#00677f", "#ADEFD1", "black"]),
    data1["years"].extend(["1995", "2020", None])

for country in zip_data_df2[zip_data_df2.Region_Name == 'Americas']['country']:
  data2["x"].extend(
      [
          zip_data_df2.loc[(zip_data_df2.year == '1995') & (zip_data_df2.country == country)]["PR"].values[0],
          zip_data_df2.loc[(zip_data_df2.year == '2020') & (zip_data_df2.country == country)]["PR"].values[0],
          None,
      ]
  )
  data2["y"].extend([country, country, None]),
  data2["colors"].extend(["#00677f", "#ADEFD1", "black"]),
  data2["years"].extend(["1995", "2020", None])

for country in zip_data_df2[zip_data_df2.Region_Name == 'Oceania']['country']:
  data3["x"].extend(
      [
          zip_data_df2.loc[(zip_data_df2.year == '1995') & (zip_data_df2.country == country)]["PR"].values[0],
          zip_data_df2.loc[(zip_data_df2.year == '2020') & (zip_data_df2.country == country)]["PR"].values[0],
          None,
      ]
    )
  data3["y"].extend([country, country, None]),
  data3["colors"].extend(["#00677f", "#ADEFD1", "black"]),
  data3["years"].extend(["1995", "2020", None])

for country in zip_data_df2[zip_data_df2.Region_Name == 'Europe']['country']:
  data4["x"].extend(
      [
          zip_data_df2.loc[(zip_data_df2.year == '1995') & (zip_data_df2.country == country)]["PR"].values[0],
          zip_data_df2.loc[(zip_data_df2.year == '2020') & (zip_data_df2.country == country)]["PR"].values[0],
          None,
      ]
  )
  data4["y"].extend([country, country, None]),
  data4["colors"].extend(["#00677f", "#ADEFD1", "black"]),
  data4["years"].extend(["1995", "2020", None])

In [ ]:
lst = ['year','CL','PR','Status']
sorted0 = zip_data_df2[zip_data_df2['Region_Name']=='Asia'].sort_values(by=['country','PR'], ascending=[False,False])
sorted1 = zip_data_df2[zip_data_df2['Region_Name']=='Africa'].sort_values(by=['country','PR'], ascending=[False,False])
sorted2 = zip_data_df2[zip_data_df2['Region_Name']=='Americas'].sort_values(by=['country','PR'], ascending=[False,False])
sorted3 = zip_data_df2[zip_data_df2['Region_Name']=='Oceania'].sort_values(by=['country','PR'], ascending=[False,False])
sorted4 = zip_data_df2[zip_data_df2['Region_Name']=='Europe'].sort_values(by=['country','PR'], ascending=[False,False])


fig1 = make_subplots(rows=1, cols=5,
                     subplot_titles=("Asia", "Africa", "Americas", "Oceania","Europe"),
                     horizontal_spacing = 0.09)

fig1.add_trace(
    go.Scatter(
    x=data0["x"],
    y=data0["y"],
    mode="lines",
    marker=dict(
        color="grey"),
    showlegend = False
    ), row=1, col=1)
fig1.add_trace(go.Scatter(
    x=data0["x"],
    y=data0["y"],
    mode="markers+text",
    marker=dict(
                color=data0["colors"],
                size=10),
    showlegend = False,
    customdata = zip_data_df2[zip_data_df2.columns & lst],
            ), row=1,col=1)

fig1.add_trace(
    go.Scatter(
    x=data1["x"],
    y=data1["y"],
    mode="lines",
    marker=dict(
        color="grey"),
    showlegend = False
    ), row=1,col=2)
fig1.add_trace(go.Scatter(
    x=data1["x"],
    y=data1["y"],
    mode="markers+text",
    marker=dict(
                color=data1["colors"],
                size=10),
    showlegend = False,
    customdata = zip_data_df2[zip_data_df2.columns & lst],
            ), row=1, col=2)

fig1.add_trace(
    go.Scatter(
    x=data2["x"],
    y=data2["y"],
    mode="lines",
    marker=dict(
        color="grey"),
    showlegend = False
    ), row=1,col=3)
fig1.add_trace(go.Scatter(
    x=data2["x"],
    y=data2["y"],
    mode="markers+text",
    marker=dict(
                color=data2["colors"],
                size=10),
    showlegend = False,
    customdata = zip_data_df2[zip_data_df2.columns & lst],
            ), row=1, col=3)

fig1.add_trace(
    go.Scatter(
    x=data3["x"],
    y=data3["y"],
    mode="lines",
    marker=dict(
        color="grey"),
    showlegend = False
    ), row=1,col=4)
fig1.add_trace(go.Scatter(
    x=data3["x"],
    y=data3["y"],
    mode="markers+text",
    marker=dict(
                color=data3["colors"],
                size=10),
    showlegend = False,
    customdata = zip_data_df2[zip_data_df2.columns & lst],
            ), row=1, col=4)

fig1.add_trace(
    go.Scatter(
    x=data4["x"],
    y=data4["y"],
    mode="lines",
    marker=dict(
        color="grey"),
    showlegend = False),
     row=1,col=5)
fig1.add_trace(go.Scatter(
    x=data4["x"],
    y=data4["y"],
    mode="markers+text",
    marker=dict(
                color=data4["colors"],
                size=10),
    showlegend = False,
    customdata = zip_data_df2[zip_data_df2.columns & lst],
            ), row=1, col=5)


## create custom legend with dummy traces
fig1.add_trace(go.Scatter(
                x=[0],
                y=[0],
                mode="markers",
                name="1995",
                marker=dict(size=13, color="#00677f", symbol='circle'),
            ))

fig1.add_trace(go.Scatter(
                x=[0],
                y=[0],
                mode="markers",
                name="2020",
                marker=dict(size=13, color="#ADEFD1", symbol='circle'),
            )) 


fig1.add_annotation(x=-0.025,y=1.12,
                    text='Political rights,<i> most free (1) to least free (7)</i>',
                    xanchor='left', yanchor='top',
                    xref='paper', yref='paper', showarrow=False,
                    font=dict(
                            family="Courier New, monospace",
                            size=18
                            ))

fig1.add_annotation(x=1,y=-0.08,
                    text='Data Source:<i> Freedom House</i>',
                    xanchor='right', yanchor='bottom',
                    xref='paper', yref='paper', showarrow=False,
                    font=dict(
                            family="Courier New, monospace",
                            size=14
                            ))

fig1.update_traces(marker_size=13)
fig1.update_xaxes(range = [0.5,7.5])    

fig1.update_yaxes(type='category',
                 matches=None, 
                 showticklabels=True,
                 dtick=1, 
                 title_font_family='Verdana',
                 title_font_size=16,
                 showgrid=False,
                 automargin=True,
                 zeroline=False 
                 )

fig1.update_layout(
    yaxis={"dtick":1})

fig1.update_layout({'plot_bgcolor': 'whitesmoke'},
                  font_family="Courier New ",
                  font_color="black",
                  title='POLITICAL FREEDOMS IN THE WORLD FROM 1995 TO 2020',
                  title_font_family='Verdana',
                  title_font_color="black",
                  title_font_size=22,
                  title_pad_b = 10,
                  legend_title_font_color="black",
                  legend_font_family='Verdana',
                  legend_title_text = '',
                  margin=dict(t=200),
                  legend=dict(yanchor='top',
                              y=1.08,
                              xanchor='left',
                              x=-0.028,
                              orientation="h"),
                   height=1000,
                   width=2200
                  )

fig1.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig1.update_xaxes(title_font_size=16,
                 title_font_family='Verdana')

fig1.update_traces(hovertemplate="<br>".join([
        "Year: %{customdata[0]}",
        "Country: %{y}",
        "Political Rights score: %{customdata[2]}",
        "Civil Liberties score: %{customdata[1]}",
        "Status: %{customdata[3]}<extra></extra>"
    ]), hoverlabel = {"font_size" : 14, "font_family" : "Courier"})

fig1.show()



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: FutureWarning:

Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: FutureWarning:

Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: FutureWarning:

Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: FutureWarning:

Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) inste

## Write up

Changes from original visual: 
1. Creating Plotly subplots for each region
2. Specified marker colors for start & end year
3. Changed default fonts, background color, axis grid lines
4. Added annotations (caption & subtitle) and custom legend
6. **Tooltips** -- customized hovertext with customdata parameter and changed font styles
